# OLCI Validation with *in situ* CCI data

    Version: 1.0
    Date:    27/09/2019
    Author:  Ben Loveday (Plymouth Marine Laboratory) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

**What is this notebook for?**

This notebook will perform a validation of a series of OLCI products using the CCI *in situ* data set. If specified, it will run the matchups according to the Bailey and Werdell (2006) protocol.

**What specific tools does this notebook use?**

HDA_API toolkit
insitu data toolkit
extraction toolkit

***

Python is divided into a series of modules that each contain a series of methods for specific tasks. The box below imports all of the moduls we need to complete our plotting task

In [ ]:
import os, sys
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import json
import scipy.stats as st
from sklearn.metrics import mean_squared_error
from zipfile import ZipFile
import warnings
warnings.filterwarnings("ignore")

# specific tools (which can be found here ../../Hub_tools/)
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/Hub_Tools/')
import harmonised_data_access_api_tools as hapi
import insitu_tools as ist
import extraction_tools as ext

WEkEO provides access to a huge number of datasets through its 'harmonised-data-access' API. This allows us to query the full data catalogue and download data quickly and directly onto our Jupyter Hub. You can search for what data is available here: https://www.wekeo.eu/dataset-navigator/start.

In order to use the HDA-API we need to provide some authentication credentials, which comes in the form of an api_key. You can get your key from here; https://www.wekeo.eu/api-keys. If you click on the 'show hidden keys' button at the bottom of the page it will reveal a number of keys. The one you need is in the top grey box, and is on the following line:

-H "Authorization: Basic "**YOUR API KEY**"

Replace "YOUR API KEY" below with what you copy from "**YOUR API KEY**" (N.B. you need to keep the quotation marks.)

In [ ]:
# your api key:
api_key = "cmJ1UGJQVzZnT09HU2RUWDJhTGFkOGY4RjhnYTpGRmFCTTNoSXluVk1NdEk4b2dPc2ZjMHFOdlVh"
# where the data should be downloaded to:
download_dir_path = "/home/jovyan/work/products"
# where we can find our data query form:
JSON_query_dir = os.path.join(os.getcwd(),'JSON_templates')
# HDA-API loud and noisy?
verbose = False

Next we will set a few parameters for selecting our data, making our plots and telling the notebook where our data is

In [ ]:
# CCI directory:
CCI_directory = 'datasets'

# select CCI in situ product: chla, iopskdtsm, rrs
insitu_product = 'rrs'

# satellite sensor
sensor = 'OLCI'

# use Bailey and Werdell, 2006 match-up criteria?
BW06 = True

# box size to extract around out points: use 2 for 5x5 box (Bailey and Werdell, 2006)
sensor_box_pad_size = 2

# select CCI in situ spectral resolution: 2, 6, fullrange
insitu_resolution = '2'

# test cases: set to None if launching your own analysis 
# available options for tests: Hawaii
test_case = 'Hawaii'

# subset region:
subset_extents = [15, 30, 55, 65]

# subset dates & times:
date_start = '2016-06-01 00:00:00'
date_end = '2019-12-31 23:59:59'

# verbosity
verbose = False
show_plots = True

# plot control
fsz = 20

# OLCI time tolerance +/- (seconds): 3 hours matches Bailey and Werdell, 2006
time_tolerance = 3*60*60 
x_tol = 0.001
flags_to_use = ['CLOUD', 'CLOUD_AMBIGUOUS', 'CLOUD_MARGIN',\
                 'INVALID', 'COSMETIC', 'SATURATED', 'SUSPECT',\
                 'HISOLZEN', 'SNOW_ICE', 'AC_FAIL', 'WHITECAPS']
flags_to_use = []

Implement test cases if we are running one...

In [ ]:
if test_case == 'Hawaii':
    # Hawaii test
    subset_extents = [-160, -155, 20, 22.5]
    date_start = '2018-10-10 00:00:00'
    date_end = '2019-12-31 23:59:59'

Now we read in and do some processing on our CCI *in situ* data

In [ ]:
insitu_headers, insitu_data = ist.read_CCI_insitu_file(CCI_directory, insitu_product, insitu_resolution)
lat_col, lon_col, time_col = ist.get_CCI_coordinate_cols(insitu_headers)
rad_cols, lambda_cols = ist.get_CCI_radiometry_cols(insitu_headers, insitu_resolution)

lats = np.asarray([float(i) for i in insitu_data[:, lat_col]])
lons = np.asarray([float(i) for i in insitu_data[:, lon_col]])
dates_times = np.asarray([datetime.datetime.strptime(i,"%Y-%m-%dT%H:%M") for i in insitu_data[:, time_col]])

Now we select the relevant data for our validation analysis

In [ ]:
# find rows that matching our subset criteria
ii = np.where((lons >= subset_extents[0]) & (lons <= subset_extents[1])\
            & (lats >= subset_extents[2]) & (lats <= subset_extents[3])\
            & (dates_times >= datetime.datetime.strptime(date_start,"%Y-%m-%d %H:%M:%S"))\
            & (dates_times <= datetime.datetime.strptime(date_end,"%Y-%m-%d %H:%M:%S")))[0]

insitu_data = insitu_data[ii,:]
lats = lats[ii]
lons = lons[ii]
dates_times = dates_times[ii]
print('-----------------------------')
print(str(len(ii)) + ' rows in scope')
print('-----------------------------')

# stop if no points in scope....
if len(ii) == 0:
    sys.exit()

We are going to bin the data spatially to help with plotting the number of observations

In [ ]:
# histogram bin the values onto a res x res grid to help adding numbers to the plot
res = 0.25
lon_grid = np.arange(int(subset_extents[0]),int(subset_extents[1])+1, res)
lat_grid = np.arange(int(subset_extents[2]),int(subset_extents[3])+1, res)
Hist, xedges, yedges = np.histogram2d(lons, lats, bins=(lon_grid, lat_grid))

# a little processing
Hist = Hist.T
Hist[Hist==0] = np.nan
plot_lon = (xedges[1:] + xedges[0:-1]) / 2
plot_lat = (yedges[1:] + yedges[0:-1]) / 2
LON, LAT = np.meshgrid(plot_lon, plot_lat)
HIST = Hist.ravel()
LON = LON.ravel()
LAT = LAT.ravel()

# tinkering with dates for plotting
dates_origin = np.asarray([tt.toordinal() for tt in dates_times])
dates_origin = (dates_origin - np.nanmin(dates_origin)) / (np.nanmax(dates_origin) - np.nanmin(dates_origin))

If verbose is set to true, we will print out our selected validation points

In [ ]:
if verbose:
    for row in range(np.shape(insitu_data)[0]):
        print('Lon: ' + str(insitu_data[row, lon_col])\
           + ' Lat: ' + str(insitu_data[row, lat_col])\
           + ' Time: ' + str(dates_times[row]))

If show_plots is set to true we will plot our validation data. The small numbers give the number of validation on a 0.25 degree grid

In [ ]:
if show_plots:
    # get our land mask from NaturalEarth
    land_resolution = '10m'
    land_poly = cfeature.NaturalEarthFeature('physical', 'land', land_resolution,\
                edgecolor='k',facecolor=cfeature.COLORS['land'])

    # intitialise our figure
    xsize = 20
    ysize = xsize*(subset_extents[3] - subset_extents[2]) / (subset_extents[1] - subset_extents[0])
    fig1 = plt.figure(figsize=(xsize, ysize), dpi=300)
    plt.rc('font', size=fsz)
    matplotlib.rcParams['contour.negative_linestyle'] = 'solid'
    
    # make an axis
    gs = gridspec.GridSpec(3, 1, height_ratios=[20,2,1])
    gs.update(wspace=0.01, hspace=0.01)
    m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree())

    # plot the data
    kk = np.argsort(dates_origin)
    plot1 = m.scatter(lons[kk], lats[kk], s=(dates_origin[kk]*-1+2)*250,
                      c=dates_origin[kk], zorder=100,
                      cmap=plt.cm.viridis, alpha=0.75,
                      transform=ccrs.PlateCarree())
    
    # embellish with gridlines and ticks
    m.coastlines(resolution=land_resolution, color='black', linewidth=1)
    m.add_feature(land_poly)
    m.set_extent(subset_extents, ccrs.PlateCarree())
    g1 = m.gridlines(draw_labels = True, zorder=20, color='0.5', linestyle='--',linewidth=0.5)
    g1.xlocator = mticker.FixedLocator(np.linspace(int(subset_extents[0]),\
                                               int(subset_extents[1]), 5))
    g1.ylocator = mticker.FixedLocator(np.linspace(int(subset_extents[2]),\
                                               int(subset_extents[3]), 5))
    g1.xlabels_top = False
    g1.ylabels_right = False
    g1.xlabel_style = {'size': fsz, 'color': 'black'}
    g1.ylabel_style = {'size': fsz, 'color': 'black'}

    # add annotation numbers
    for hh in range(len(HIST)):
        if np.isnan(HIST[hh]):
            continue      
        else:
            dx = (subset_extents[1] - subset_extents[0])/80
            dy = (subset_extents[3] - subset_extents[2])/80
            plt.annotate(str(HIST[hh]), xy=(LON[hh]+dx, LAT[hh]+dy),\
                         xycoords='data', size=fsz,\
                         color='r', zorder=100)
    
    # add colorbar
    axes0 = plt.subplot(gs[2,0])
    cbar = plt.colorbar(plot1, cax=axes0, orientation='horizontal',ticks=[0, 1])
    cbar.ax.set_xticklabels([min(dates_times), max(dates_times)])
    cbar.ax.tick_params(labelsize=fsz) 
    cbar.set_label('Measurement date & time',fontsize=fsz)

Now we will build a series of queries that check if any OLCI passes match with this data. We start by setting the data source (which we use as a key for our JSON query file)

In [ ]:
# CMEMS OSTIA SST KEY
dataset_id = "EO:EUM:DAT:SENTINEL-3_OL_2_WFR___"

Set our date requirements using our in situ validation points and OLCI time tolerance

In [ ]:
start_dates = []
end_dates = []
for this_date in dates_times:
    start_dates.append((this_date - datetime.timedelta(seconds=time_tolerance)).strftime("%Y-%m-%dT%H:%M:%S.000Z"))
    end_dates.append((this_date + datetime.timedelta(seconds=time_tolerance)).strftime("%Y-%m-%dT%H:%M:%S.000Z"))

if verbose:
    for start_date, end_date in zip(start_dates, end_dates):
        print(start_date, end_date)

We use our data source to load the correct JSON query file from ../JSON_templates. (This is just my convention for building queries, there are many other ways you could do this).

In [ ]:
# find query file
JSON_query_file = os.path.join(JSON_query_dir,dataset_id.replace(':','_')+".json")
if not os.path.exists(JSON_query_file):
    print('Query file ' + JSON_query_file + ' does not exist')
else:
    print('Found JSON query file for '+dataset_id)

Now we download the data. See the following scripts and notebooks for information on how this works:
    
    *samples/How_To_Guide-Harmonized_Data_Access-v0.1.3.ipynb*
    *ocean-wekeo-jpyhub/HDA_API_Tools/HDA_API_downloading.ipynb*
    *ocean-wekeo-jpyhub/Hub_Tools/harmonised_data_access_api_tools.py*

In [ ]:
HAPI_dict = hapi.init(dataset_id, api_key, download_dir_path, verbose=verbose)
HAPI_dict = hapi.get_access_token(HAPI_dict)
HAPI_dict = hapi.accept_TandC(HAPI_dict)

Downloaded_files = []
Downloaded_iter = []

count = -1
for start_date, end_date in zip(start_dates, end_dates):
    count = count + 1
    print(str(int(count/len(start_dates)))+'% complete')
    print('Running for: ' + start_date + ' to ' + end_date)

    date_string = start_date.split('T')[0].replace('-','') \
                  + '_' + end_date.split('T')[0].replace('-','')
    
    # load the query
    with open(JSON_query_file, 'r') as f:
        query = f.read()
        query = query.replace("%DATE_START%",start_date)
        query = query.replace("%DATE_END%",end_date)
        query = query.replace("%LON1%",str(lons[count] - x_tol))
        query = query.replace("%LON2%",str(lons[count] + x_tol))
        query = query.replace("%LAT1%",str(lats[count] - x_tol))
        query = query.replace("%LAT2%",str(lats[count] + x_tol))
        query = json.loads(query)
    
    # launch job
    HAPI_dict = hapi.launch_query(HAPI_dict, query)

    # wait for jobs to complete
    HAPI_dict = hapi.check_job_status(HAPI_dict)
    if HAPI_dict['nresults'] == 0:
        print('Nothing to do for this query....')
        continue

    # check results and links
    HAPI_dict = hapi.get_results_list(HAPI_dict)
    HAPI_dict = hapi.get_download_links(HAPI_dict)

    # check prospective filenames:
    HAPI_dict = hapi.get_filenames(HAPI_dict)

    # download
    HAPI_dict = hapi.download_data(HAPI_dict, skip_existing=True)
    
    for file in HAPI_dict['filenames']:
        Downloaded_files.append(file)
        Downloaded_iter.append(count)

OLCI files are downloaded in the zipped SAFE format. We have to unzip these files to use them, which we do below.

In [ ]:
# unzip file
for filename in Downloaded_files:
    if os.path.splitext(filename)[-1] == '.zip':
        print('Unzipping: ' + filename)
        with ZipFile(filename, 'r') as zipObj:
            # Extract all the contents of zip file in current directory
            zipObj.extractall(os.path.dirname(filename))

Now we compare the downloaded files with the *in situ* record

In [ ]:
# open relevant files and get validation points
all_insitu_vals = []
all_OLCI_vals = []
all_band_wavs = []
for ii in range(len(Downloaded_iter)):
    input_dir = Downloaded_files[ii].replace('.zip','.SEN3')
    band_wavs, OLCI_vals = \
       ext.extract_OLCI_matchup_data(input_dir,\
                            lats[Downloaded_iter[ii]],\
                            lons[Downloaded_iter[ii]],\
                            insitu_product,\
                            pad_size=sensor_box_pad_size,\
                            flags_to_use=flags_to_use, BW06=BW06)
    insitu_vals = \
       ext.extract_CCI_matchup_data(insitu_product,\
                            insitu_data[Downloaded_iter[ii],:],\
                            rad_cols)
        
    # Normalise the OLCI reflectances for BDRF
    if BW06:
        OLCI_means = np.asarray([record['filtered_var_mean'] for record in OLCI_vals]).astype(float)
        OLCI_CV = np.asarray([record['CV'] for record in OLCI_vals]).astype(float)
        if np.median(OLCI_CV > 0.15):
            OLCI_means = OLCI_means*np.nan
    else:
        OLCI_means = np.asarray(OLCI_vals.copy()).astype(float)
        
    BDRF_norm_array = OLCI_means/np.pi
    
    # write into list; truncating to the in situ wavelengths
    all_insitu_vals.append(np.asarray(insitu_vals))
    all_OLCI_vals.append(BDRF_norm_array[0:len(insitu_vals)])
    all_band_wavs.append(np.asarray(band_wavs[0:len(insitu_vals)]))

Now plot the matchup data by spectra

In [ ]:
# set up figure
fig1 = plt.figure(figsize=(len(all_band_wavs)*5, 10), dpi=300)
plt.rc('font', size=fsz)

# setup axes
gs = gridspec.GridSpec(3, 1)
gs.update(hspace=0.5)

for ii in range(len(all_band_wavs)): 
    ax = plt.subplot(gs[ii,0])
    plt.plot(all_band_wavs[ii], all_OLCI_vals[ii], color='r', linestyle='--')
    plt.plot(all_band_wavs[ii], all_insitu_vals[ii], color='b', linestyle='--')
    p1 = plt.scatter(all_band_wavs[ii], all_OLCI_vals[ii], color='r', s=50)
    p2 = plt.scatter(all_band_wavs[ii], all_insitu_vals[ii], color='b', s=50)
    plt.xlabel('Wavelength [nm]')
    plt.ylabel('Rrs [sr$^{-1}$]')
    plt.ylim([0, np.nanmax(all_OLCI_vals)*1.25])
    leg = plt.legend([p1, p2],['OLCI','CCI'])
    leg.get_frame().set_linewidth(0.0)

plt.show()

Now plot the matchup data by wavelength

In [ ]:
# set up panels
ncols = 3
nrows = (np.ceil(len(all_band_wavs[0])/ncols)).astype('int')

# set up figure
fig1 = plt.figure(figsize=(ncols*10, nrows*10), dpi=300)
plt.rc('font', size=fsz)

# set up axes
gs = gridspec.GridSpec(nrows, ncols)
gs.update(wspace=0.25, hspace=0.25)

count = -1
for ii in range(nrows):
    for jj in range(ncols):
        count = count + 1
        if count >= len(all_band_wavs[0]):
            continue
        # Plot the matchups
        ax = plt.subplot(gs[ii,jj])
        this_wavs = [] ; this_insitu = [] ; this_olci = []
        for kk in range((len(all_band_wavs))):
            this_wavs.append(all_band_wavs[kk][count])
            this_insitu.append(all_insitu_vals[kk][count])
            this_olci.append(all_OLCI_vals[kk][count])
        
        this_wavs = np.asarray(this_wavs)
        this_insitu = np.asarray(this_insitu)
        this_olci = np.asarray(this_olci)
        
        p1 = plt.scatter(this_insitu, this_olci, color='r', s=50)
        plt.xlabel('Rrs in situ [sr$^{-1}$]')
        plt.ylabel('Rrs OLCI [sr$^{-1}$]')
        max_val = max([np.nanmax(this_insitu), np.nanmax(this_olci)])*1.1
        plt.xlim([0, max_val])
        plt.ylim([0, max_val])
        
        # embellish
        plt.title('Wavelength: '+str(this_wavs[0])+' nm')
        plt.plot([0, max_val], [0, max_val], linestyle='--', color='k')
        
        # stats
        slp, inc, rval, pval, stderr = st.linregress(this_insitu, y=this_olci)
        rmse = mean_squared_error(this_insitu, this_olci)**0.5

        plt.annotate('slope: {0:.4f}'.format(slp), xy=(0.05, 0.95), xycoords='axes fraction', color='b')
        plt.annotate('intercept: {0:.4f}'.format(inc), xy=(0.05, 0.90), xycoords='axes fraction', color='b')
        plt.annotate('rval: {0:.4f}'.format(rval), xy=(0.05, 0.85), xycoords='axes fraction', color='b')
        plt.annotate('pval: {0:.4f}'.format(pval), xy=(0.05, 0.80), xycoords='axes fraction', color='b')
        plt.annotate('rmse: {0:.4f}'.format(rmse), xy=(0.05, 0.75), xycoords='axes fraction', color='b')
        plt.plot([0, max_val], [0*slp+inc, max_val*slp+inc], linestyle='--', color='r')